# Dumping Data

### dumping 2.0 data
have to break it 2 parts because mysql killed the long process
- Part 1
```
mysql -h gs-read-2016-3-24-1.cpkpnksgl3uc.us-west-1.rds.amazonaws.com -u inka -pcharleebear gigspoon -e "select concat(agent_id,',', agent_salt, ',',agent_saltedPasswordHash, ',',agent_email_address,',' ,agent_create_timestamp,',',agent_reputation, ',',agent_structured_data) from agent_accounts where agent_account_state = 'APPROVED' order by agent_id limit 700000" > gigwalkers_20180111.csv
```

- Part 2
```
mysql -h gs-read-2016-3-24-1.cpkpnksgl3uc.us-west-1.rds.amazonaws.com -u inka -pcharleebear gigspoon -e "select concat(agent_id,',', agent_salt, ',',agent_saltedPasswordHash, ',',agent_email_address,',' ,agent_create_timestamp,',',agent_reputation, ',',agent_structured_data) from agent_accounts where agent_account_state = 'APPROVED' order by agent_id limit 700000, 1296000" > gigwalkers_20180111_2.csv
```
- copy(scp) it to local machine & remove first line of part2 data header

- Concat them
```
cat gigwalkers_20180111.csv gigwalkers_20180111_2.csv > gigwalkers_2018.csv
```

- getting 3.0 data, change sql as needed
```
psql -U derek -d gw_apps_1 -p 5432 -h new-prod.cgxxy7nmhwzi.us-east-1.rds.amazonaws.com -c                             
"Copy (SELECT agent_id from customer_profile where agent_id is not NULL) To STDOUT With CSV HEADER DELIMITER ',';" > 3.0agents_data.csv
```


# Cleaning up 20180109.csv

In [1]:
import csv
import json
import pandas as pd
import numpy as np
from datetime import datetime 
from pandas.io.json import json_normalize
import os

In [ ]:
rs = []
df = pd.DataFrame()
result_df = []
# open the csv file
with open('gigwalkers_2018.csv') as fh:
    rows = csv.reader(fh, delimiter=',', quoting=csv.QUOTE_NONE)
    header = next(rows)
    for i, row in enumerate(rows):
        # print row
        # print len(row)
        # last few columns are json structure data
        js= ','.join(row[6:])
        js = js.replace("\\\\", "\\")
        # print json.loads(js)
        
        rec = (row[0], row[1], row[2], row[3], row[4], row[5], json.loads(js))
        sr = pd.Series(data=rec[0:-1])
        sr= sr.append(json_normalize(rec[-1]).transpose())
        df_rec = sr.transpose()
        # print df_rec
        # df_rec will be a datafram row
        df = df.append(df_rec)
        if (i+1)%1000 == 0:
            # saving every 1000 rec. to a pickle file
            df.to_pickle('temp/temp{}.pkl'.format(i+1))
            # reset df to empty
            df = pd.DataFrame()
            # print i
            # extra steps, not really need
            # result_df.append(df)
    if len(df):
        df.to_pickle('temp/temp{}.pkl'.format('_last'))

In [ ]:
# loading from all files
arr = os.listdir('temp/')
l = [pd.read_pickle('temp/'+pkl) for pkl in arr]
df = pd.DataFrame()
df = pd.concat(l)

# saving to one pickle file
df.to_pickle('gigwalkers_2018.pkl')

In [2]:
df = pd.read_pickle('gigwalkers_2018.pkl')
# dropping existing index, coz it's all 0
df = df.reset_index(drop=True)
# rename the column names 
df.rename(columns={0:'agent_id', 1:'agent_salt', 2:'agent_saltedPasswordHash', 3:'email', 4: 'time_creaeted',5:'reputation'}, inplace=True)

In [3]:
# cleaning None values
df.replace('none', np.NaN, inplace=True)
df.replace('N/A', np.NaN, inplace=True)

# cleaning Yes/No values
df.replace('Yes', True, inplace=True)
df.replace('yes', True, inplace=True)
df.replace('no', False, inplace=True)
df.replace('False', False, inplace=True)

In [4]:
df['reputation'] = df['reputation'].apply(float)

In [5]:
# clean location_time_updated, cannot use int because there's nan data
# df['location_time_updated']= df['location_time_updated'].apply(float)
df['location_time_updated'] = pd.to_datetime(df.location_time_updated, unit='s')

In [6]:
# clean lat, 
# df['lat'].apply(float) might fail, but it's a good idea to run first to know what data look like
# mask for lat has some weird strin
# mask = df['lat'] == u'??????????'
# mask = df['lat'].isnull()
df['lat'] = df['lat'].str.replace(',','.')
df['long'] = df['long'].str.replace(',','.')
df.lat = pd.to_numeric(df['lat'], errors='coerce')

# clean long ?1.4831493, ?0.7722991, ?1.1991791, ?111.8050423
# mask = df['long'] == '?111.8050423'
# mask = df['long'] == '????????????'
# mask = df['long'] == '???????'
# mask = df['long'] == '?-?????????'

# df.loc[mask, 'long'] = np.NaN
# df['long'].apply(float)

df.long = pd.to_numeric(df['long'], errors='coerce')

In [7]:
# cleaning gender
df.loc[df['gender'] == 'male', 'gender'] = 'Male'
df.loc[df['gender'] == 'female', 'gender'] = 'Female'

In [8]:
# transform birthday
df['birthday'] = pd.to_datetime(df.birthday, unit='s')
# transform time_creaeted
df['time_creaeted'] = pd.to_datetime(df.time_creaeted, unit='s')

In [9]:
# clean up `agent_avatar`
df['agent_avatar'].replace(False, np.NaN, inplace=True)
df['agent_avatar'].replace('[object Object]', np.NaN, inplace=True)

df.loc[df['agent_avatar'].notna(), 'agent_avatar'] = df.loc[df['agent_avatar'].notna(), 'agent_avatar'].apply(lambda x: x if '.jpg' in x else x+'.jpg')

In [10]:
# clean up facebook_id
df['facebook_id'].replace(False, np.NaN, inplace=True)
df['facebook_id'].replace('0', np.NaN, inplace=True)
df.facebook_id = pd.to_numeric(df['facebook_id'], errors='coerce')

In [11]:
# change min_payout and rating to numeric value
df.min_payout = pd.to_numeric(df['min_payout'], errors='coerce')
df.rating = pd.to_numeric(df['rating'], errors='coerce')

In [12]:
# clean profile_picture_id
df['profile_picture_id'] = df['profile_picture_id'].replace(False, None)
df.loc[(df['profile_picture_id'] == False),'profile_picture_id'] = None

In [13]:
# these are varchar2 should not be stamped as 'False'
df['udid'] = df['udid'].replace(False, np.NaN)
df['device_type'] = df['device_type'].replace(False, np.NaN)
df['device_os'] = df['device_os'].replace(False, np.NaN)

In [14]:
df.to_pickle('gigwalkers_2018_cleaned.pkl')

In [ ]:
# ignoring columns: 
# current_city_location_id, in 3.0 wrongly designed as integer, where data is mostly string
# metro_area_location_id, in 3.0 wrongly designed as integer, where data is mostly string
# email_address, 9 different, we can ignore
# agent_skills, looks like a dictionary

In [ ]:
df.info()

# 3.0 dataframe and comparing with 2.0 data

In [ ]:
df = pd.read_pickle('gigwalkers_2018_cleaned.pkl')
df3 = pd.read_csv('3.0agents_data.csv')
df_merge=pd.merge(df,df3,on=['agent_id'],how="outer",indicator=True)
need_import = df_merge[df_merge['_merge'] == 'left_only']
need_import = need_import.where((pd.notnull(need_import)), None)

# Import to 3.0

In [ ]:
# import, so obviously these code need to be run under gigwalk app
import pandas as pd
import numpy as np
from sqlalchemy_utils.types.phone_number import PhoneNumber
from gigwalk_api_app.customers.models import Customer
from gigwalk_api_app.customer_profile.models import CustomerProfile
from datetime import datetime

In [ ]:
# setup
df = pd.read_pickle('gigwalkers_2018_cleaned.pkl')
df3 = pd.read_csv('3.0agents_data.csv')
df_merge=pd.merge(df,df3,on=['agent_id'],how="outer",indicator=True)
need_import = df_merge[df_merge['_merge'] == 'left_only']
need_import = need_import.where((pd.notnull(need_import)), None)

In [ ]:
lp = LocationProxy(app.config)
def map_avatar(avatar):
    # sometimes agent_avatar is False so we have to replace that with the default image
    if not avatar:
        return 'http://www.gigwalk.com/img/default-user-image.gif'
    else:
        return 'https://s3-us-west-1.amazonaws.com/gigwalk-profile-photos/{}'.format(avatar)

def create_customer(agent):
    email = agent.get('email')
    customer = Customer.get_by_email(email)
    # Create new user if does not exists
    if customer:
        return
    
    customer = Customer()
    customer.first_name = agent.get('first_name')
    customer.last_name = agent.get('last_name')
    customer._password = agent.get('agent_saltedPasswordHash').encode('utf8')
    customer._salt = agent.get('agent_salt')
    customer.photo_url = map_avatar(agent.get('agent_avatar'))
    customer.customer_status = 'ACTIVE'
    customer.email = agent.get('email')
    customer.date_created = agent.get('agent_create_timestamp')
    customer.organization_id = 5
    customer.role = 'WORKER'
    customer.home_latitude = agent.get('lat')
    customer.home_longitude = agent.get('long')
    customer.date_updated = agent.get('location_time_updated')
    
          
    # if not customer.profile:
    profile_keys = [u'rating', u'device_type', u'skills_hotel', u'agent_skills', u'profile_picture_id', u'education_level', u'push_notification_device_token', u'occupation', u'skills_writing', u'paypal_email_address', u'skills_real_estate', u'facebook_id', u'min_payout', u'phone_number', u'bio', u'skills_retail', u'last_place_employed_JSON', u'facebook_access_token', u'skills_restaurant', u'birthday', u'device_os', u'skills_photography', u'udid', u'gender', u'APIHash', 'reputation', u'agent_avatar', 'agent_id']
    customer.profile = CustomerProfile()
    customer.profile.date_created = datetime.now()    
    for k, v in agent.items():
        if k in profile_keys:
            setattr(customer.profile, k, v)

    # set tzid and home address
    if customer.home_latitude and customer.home_longitude:
        try:
        	print "tzid"
            # customer.tzid = TZWorld.tzid_from_coordinates(customer.home_longitude, customer.home_latitude)
        except Exception as e:
            # Probably a glitch, just ignore and continue
            print("Error {} in calculating timezone while importing {}".format(e, customer.email))
        try:
            loc = lp.reverse_geocode(customer.home_latitude, customer.home_longitude)
        except Exception as e:
            # Probably some geocoding glitch, just ignore and go to the next customer
            print("Geocoding error {} while importing {}".format(e, customer.email))

        if loc:
            acs = loc.raw.get('address_components')
            street_number = None
            route = None
            for ac in acs:
                loc_type = str(ac.get('types')[0])
                if loc_type == 'subpremise':
                    customer.address_line_2 = ac.get('short_name')
                elif loc_type == 'street_number':
                    street_number = ac.get('short_name')
                elif loc_type in ['street_address', 'premise']:
                    customer.address_line_1 = ac.get('short_name')
                elif loc_type == 'route':
                    route = ac.get('short_name')
                elif loc_type == 'postal_code':
                    customer.postal_code = ac.get('short_name')
                elif loc_type == 'locality':
                    customer.city = ac.get('short_name')
                elif loc_type == 'administrative_area_level_1':
                    customer.state = ac.get('short_name')
                elif loc_type == 'country' and customer.profile.phone_number:
                    country = ac.get('short_name')
                    customer.phonenumber = PhoneNumber(customer.profile.phone_number, country)
            if not customer.address_line_1 and route:
                customer.address_line_1 = street_number + ' ' + route if street_number else route
          
    customer.save()

In [ ]:
for index, row in need_import.head(3).iterrows():
    create_customer(dict(row))

In [ ]:
# this is using gmaps API to do geocoding, should NOT use it for 1.2M gigwalkers
# let's checkout the following link for state, county search, or maybe a radius search could work
# https://github.com/thampiman/reverse-geocoder 

def find_zipcode(lat_long):
    lat, longti = lat_long.split(',')
    reverse_geocode_result = gmaps.reverse_geocode((lat, longti))
    if reverse_geocode_result:
        components = reverse_geocode_result[0]['address_components']
        for comp in components:
            if 'postal_code' in comp['types']:
                return int(comp['short_name'])
    else:
        return None
    
